# Terminology mapping

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
def runQuery(endpointUrl, query):
    sparql = SPARQLWrapper(endpointUrl)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [ ]:
repository = "http://sparql_open:7200/repositories/data"

queryLocalTerms = """PREFIX roo: <http://www.cancerdata.org/roo/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?type ?value
from <http://data.local/rdf>
WHERE {
	?obj rdf:type ?type.
    ?obj roo:local_value ?value .
    FILTER(?type NOT IN (owl:NamedIndividual)).
}
ORDER BY ?type"""

# get local values
localValues = runQuery(repository, queryLocalTerms)
for localValue in localValues["results"]["bindings"]:
    print(localValue["type"]["value"] + ": " + localValue["value"]["value"])

In [ ]:
import rdflib
g = rdflib.Graph()
g.load("https://raw.githubusercontent.com/RadiationOncologyOntology/ROO/master/owl/ROO.owl")

In [ ]:
for localValue in localValues["results"]["bindings"]:
    myQuery = "SELECT ?label WHERE { <%s> rdfs:label ?label. }" % (localValue["type"]["value"])
    for row in g.query(myQuery):
        print(row.label + ": " + localValue["value"]["value"])

In [ ]:
import ipywidgets as widgets
import collections
myWidgets = [ ]
myButtonList = dict()

def save_reasoning(localValue, superClass, term):
    query = """
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX roo: <http://www.cancerdata.org/roo/>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        
        INSERT {
        GRAPH <http://data.local/mapping> {
            <%s> owl:equivalentClass [
                rdf:type owl:Class;
                owl:intersectionOf [
                    rdf:first <%s>;
                    rdf:rest [
                        rdf:first [
                            rdf:type owl:Class;
                            owl:unionOf [
                                rdf:first [
                                    rdf:type owl:Restriction;
                                    owl:hasValue "%s"^^xsd:string;
                                    owl:onProperty roo:local_value;
                                ];
                                rdf:rest rdf:nil;
                            ]
                        ];
                        rdf:rest rdf:nil;
                    ]
                ]
            ].
        } } WHERE { }
    """ % (term, superClass, localValue)
    print(query)
    
    sparql = SPARQLWrapper(repository + "/statements")
    sparql.setQuery(query)
    sparql.method = "POST"

    try:
        sparql.query()
        print("Mapping saved.")
    except:
        print("Something went wrong during the update. Please check if the RDF repository is available.")

def save_action(b):
    myButtonObjects = myButtonList[b]
    
    #loop over all entries for this dropdown, to find the correct ID
    uri = None
    label = None
    for entry in myButtonObjects["classList"]:
        if(entry["id"] == myButtonObjects["dropdown"].value):
            uri = entry["uri"]
            label = entry["label"]
    
    if (uri is not None) & (label is not None):
        save_reasoning(myButtonObjects["localValue"], myButtonObjects["superClass"], uri)

#loop over all found local values
for localValue in localValues["results"]["bindings"]:
    superClass = localValue["type"]["value"]
    localValue = localValue["value"]["value"]
    
    # check the rdfs:label for all found superclasses
    myQuery = "SELECT ?label WHERE { <%s> rdfs:label ?label. }" % superClass
    superClassName = ""
    for row in g.query(myQuery):
        superClassName = row.label
    
    # create textfield with the local value as text, superclass label as description
    valueWidget = widgets.Text(value=localValue, disabled=True, description=superClassName, style={"description_width": "initial"})
    
    # lookup subclasses (possible matching candidates)
    myQuery = """
        SELECT ?subClass ?label 
        WHERE { 
            ?subClass rdfs:subClassOf* <%s>.
            ?subClass rdfs:label ?label.
            
            FILTER (?subClass != <%s>).
        }
        ORDER BY ?label""" % (superClass, superClass)
    queryResult = g.query(myQuery)
    
    # make list for dropdown
    classList = list()
    i=0
    for subClassResultRow in queryResult:
        classList.append({"uri": str(subClassResultRow.subClass), "label": str(subClassResultRow.label), "id": i})
        i = i+1
    
    dropdownValues = dict()
    selectedId = 0
    for myClass in classList:
        dropdownValues[myClass["label"]] = myClass["id"]
        ##add here to set the value

    dropdownValues = collections.OrderedDict(sorted(dropdownValues.items()))
    
    # create dropdown
    standardValWidget = widgets.Dropdown(
        options=dropdownValues,
        value=selectedId
    )
    
    # create save button
    saveWidget = widgets.Button(
        description="Save",
        icon='save',
    )
    
    myButtonList[saveWidget] = {"superClass": superClass, "localValue": localValue, "dropdown": standardValWidget, "classList": classList}
    saveWidget.on_click(save_action)
    
    #align all three widget objects
    myWidgets.append(widgets.HBox([valueWidget, standardValWidget, saveWidget]))

#align all entries
widgets.VBox(myWidgets)